<a href="https://colab.research.google.com/github/Unknown52790/Sales-Prediction/blob/main/Tiles_Color_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Importing the Libraries***

In [6]:
from PIL import Image
import os
import numpy as np
import cv2
import webcolors

# ***Check The Background Color***

In [7]:
def get_background_color(image_path):
    try:
        # Open the image
        img = Image.open(image_path)

        # Convert the image to RGB mode if it's not already in RGB
        img = img.convert("RGB")

        # Get the most frequent color in the image
        background_color = img.getpixel((0, 0))

        return background_color
    except IOError:
        print(f"Error: Unable to open image '{image_path}'.")



# ***Get the Pattern Color***

In [8]:
def get_pattern_color(image_path, roi):
    try:
        # Open the image
        img = Image.open(image_path)

        # Convert the image to RGB mode if it's not already in RGB
        img = img.convert("RGB")

        # Get the average color within the specified region of interest (ROI)
        cropped_img = img.crop(roi)
        pattern_color = cropped_img.resize((1, 1)).getpixel((0, 0))

        return pattern_color
    except IOError:
        print(f"Error: Unable to open image '{image_path}'.")
    except Exception as e:
        print(f"Error: {e}")

# ***Check the Thickness of the Tile Pattern***

In [9]:
def classify_pattern_thickness(image_path, roi, threshold):
    try:
        # Open the image
        img = Image.open(image_path)

        # Get the width of the pattern within the specified region of interest (ROI)
        pattern_width = roi[2] - roi[0]

        # Classify the pattern as thick or thin based on the width
        pattern_thickness = "thick" if pattern_width >= threshold else "thin"

        return pattern_thickness
    except IOError:
        print(f"Error: Unable to open image '{image_path}'.")
    except Exception as e:
        print(f"Error: {e}")

# ***ROI (Region of Interest)***

In [10]:
def detect_roi(image_path):
    try:
        # Read the image using OpenCV
        img = cv2.imread(image_path)

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Perform edge detection using Canny
        edges = cv2.Canny(blurred, 50, 150)

        # Find contours in the edge-detected image
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Get the largest contour (assuming it's the marble pattern)
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Define the ROI based on the bounding box
        roi = (x, y, x+w, y+h)

        return roi
    except Exception as e:
        print(f"Error: {e}")

# ***Rename that Particular Tile with Color and Thickness***

In [11]:
def rename_with_colors_and_thickness(image_path, threshold):
    try:
        background_color = get_background_color(image_path)
        roi = detect_roi(image_path)
        pattern_color = get_pattern_color(image_path, roi)
        pattern_thickness = classify_pattern_thickness(image_path, roi, threshold)

        # Convert RGB colors to string representation
        background_color_name = get_color_name(background_color)
        pattern_color_name = get_color_name(pattern_color)

        # Construct new file name with background color, pattern color, and thickness
        new_file_name = f"{background_color_name}_{pattern_color_name}_{pattern_thickness}"

        # Get the file extension
        _, extension = os.path.splitext(image_path)

        # Rename the file
        os.rename(image_path, new_file_name + extension)

        print(f"File '{image_path}' has been renamed to '{new_file_name}{extension}'.")
    except Exception as e:
        print(f"Error: {e}")
def color_distance(color1,color2):
  return sum((c1-c2)**2 for c1,c2 in zip(color1,color2))**0.5

# ***Display Output***

In [13]:
def get_color_name(color):
  try:
    color_name=webcolors.rgb_to_name(color)
  except ValueError:
    closest_color=min(webcolors.CSS3_NAMES_TO_HEX,key=lambda x:color_distance(color,webcolors.hex_to_rgb(webcolors.CSS3_NAMES_TO_HEX[x])))
    color_name=closest_color.replace('-','').title()
  return color_name

# Example usage:
image_path = "/content/Brown.jpg"
threshold = 10  # Example threshold value for pattern thickness classification
rename_with_colors_and_thickness(image_path, threshold)

File '/content/Brown.jpg' has been renamed to 'Linen_Rosybrown_thick.jpg'.
